## **Scraping data about ebook readers from amazon**

## Libraries

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import seaborn as sns
from selenium.webdriver.support import expected_conditions as EC

## Code for getting the links

### Note: scroll_to_bottom() is a function that forces the page to scroll down to the bottom. It was needed because the code had problems with collecting all items within the page


In [51]:
s = Service('C:\webdrivers\chromedriver.exe')
driver = webdriver.Chrome(service=s)
page_url = "https://www.amazon.pl/gp/bestsellers/electronics/20788326031/ref=zg_bs_pg_2_electronics?ie=UTF8&pg=1"
driver.get(page_url)
driver.implicitly_wait(5)
cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//span[@class="a-button-inner"]//input[@id="sp-cc-accept"]')))
cookies.click()
all_links = []

def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3) 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


for page in range(1,3):
    scroll_to_bottom()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="zg-grid-general-faceout"]')))
    links = driver.find_elements(By.XPATH,'//div[@class="zg-grid-general-faceout"]//div[@class="p13n-sc-uncoverable-faceout"]//a[@class="a-link-normal"][2]')
    for link in links:
        link_url = link.get_attribute('href')
        all_links.append({'link':link_url})
    if page == 1:
        next_page = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//li[@class="a-last"]//a')))
        next_page.click()
        time.sleep(10)

for link_info in all_links:
    print(link_info['link'])

https://www.amazon.pl/kindle-paperwhite-16-gb-teraz-z-wyswietlaczem-68-i-regulowanym-podswietleniem-w-cieplym-kolorze-bez-reklam/dp/B09TMF6742/ref=zg_bs_g_20788326031_sccl_1/257-1810756-1248753?psc=1
https://www.amazon.pl/Kindle-2022/dp/B09SWS16W6/ref=zg_bs_g_20788326031_sccl_2/257-1810756-1248753?psc=1
https://www.amazon.pl/kindle-paperwhite-signature-edition-32-gb-z-wyswietlaczem-68-bezprzewodowym-ladowaniem-i-automatycznie-regulowanym-przednim-podswietleniem/dp/B08N2QK2TG/ref=zg_bs_g_20788326031_sccl_3/257-1810756-1248753?psc=1
https://www.amazon.pl/Kindle-2022/dp/B09SWTJZH6/ref=zg_bs_g_20788326031_sccl_4/257-1810756-1248753?psc=1
https://www.amazon.pl/Kindle-regulacj%C4%85-pod%C5%9Bwietlenia-wodoodporny-grafitowy/dp/B07L5GDTYY/ref=zg_bs_g_20788326031_sccl_5/257-1810756-1248753?psc=1
https://www.amazon.pl/Kindle-Oasis-regulacj%C4%85-pod%C5%9Bwietlenia-wodoodporny/dp/B07L5K4TG3/ref=zg_bs_g_20788326031_sccl_6/257-1810756-1248753?psc=1
https://www.amazon.pl/Pocketbook-Verse-Czytnik-Boo

In [52]:
len(all_links)

69

In [53]:
all_links_fixed = []
for k in all_links:
    all_links_fixed.append(k['link'])

## Code for getting specific data about the products from each previously scraped link

In [55]:
product_data = []
for each_link in all_links_fixed:
    driver.get(each_link)
    driver.implicitly_wait(5)
    product_link = each_link
    product_title = driver.find_element(By.XPATH,'//span[@id="productTitle"]').text.strip()
    try:
        product_note = driver.find_element(By.XPATH,'(//span[@class="a-size-base a-color-base"])[2]').text.strip()
    except:
        product_note = np.NaN
    try:
        product_price = driver.find_element(By.XPATH,'//span[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]').text.strip()
    except:
        product_price = np.NaN
    try:
        product_nr_reviews = driver.find_element(By.XPATH,'//span[@id="acrCustomerReviewText"]').text.strip()
    except:
        product_nr_reviews = np.NaN
    product_data.append({'link':product_link,
                         'title':product_title,
                         'rate':product_note,
                         'price':product_price,
                         'nr_reviews':product_nr_reviews})
    time.sleep(2)

In [58]:
df = pd.DataFrame(product_data)

In [60]:
df

,link,title,rate,price,nr_reviews
0,https://www.amazon.pl/kindle-paperwhite-16-gb-...,Kindle Paperwhite (16 GB) – teraz z wyświetlac...,"4,8",789\n99zł,Liczba ocen: 753
1,https://www.amazon.pl/Kindle-2022/dp/B09SWS16W...,Kindle (2022) — najlżejszy i najbardziej kompa...,"4,7",509\n99zł,Liczba ocen: 147
2,https://www.amazon.pl/kindle-paperwhite-signat...,Kindle Paperwhite Signature Edition (32 GB) – ...,"4,8",869\n99zł,Liczba ocen: 753
3,https://www.amazon.pl/Kindle-2022/dp/B09SWTJZH...,Kindle (2022) — najlżejszy i najbardziej kompa...,"4,7",509\n99zł,Liczba ocen: 147
4,https://www.amazon.pl/Kindle-regulacj%C4%85-po...,Kindle Oasis | z regulacją barwy podświetlenia...,"4,5",999\n99zł,Liczba ocen: 58
...,...,...,...,...,...
64,https://www.amazon.pl/przeno%C5%9Bny-wy%C5%9Bw...,"Czytnik e-booków, przenośny 15 cm wyświetlacz ...","1,9",472\n92zł,Liczba ocen: 2
65,https://www.amazon.pl/Tolino-4016621126872-Czy...,"Tolino Shine 3 4016621126872 Czytnik, Czarny, ...","4,2",NaN,Liczba ocen: 848
66,https://www.amazon.pl/Czytnik-book%C3%B3w-Kr%C...,Czytnik e-booków Krüger & Matz Library 3S CART...,"3,0",NaN,Liczba ocen: 12
67,https://www.amazon.pl/PocketBook-Czytnik-book%...,PocketBook Czytnik e-booków InkPad 4 (wersja n...,,1 519\n00zł,Liczba ocen: 2


## Data cleaning

In [75]:
df['price'] = df['price'].apply(lambda x: x.strip().replace('\n','.').replace('zł','').strip() if type(x) == str else x)

In [79]:
df.iloc[:,-1] = df.iloc[:,-1].apply(lambda x: x.strip().replace('Liczba ocen: ','').strip() if type(x) == str else x)

In [84]:
df['rate'] = df['rate'].apply(lambda x :x.strip().replace(',','.').strip() if type(x) == str else x)

In [85]:
df.head()

,link,title,rate,price,nr_reviews
0,https://www.amazon.pl/kindle-paperwhite-16-gb-...,Kindle Paperwhite (16 GB) – teraz z wyświetlac...,4.8,789.99,753
1,https://www.amazon.pl/Kindle-2022/dp/B09SWS16W...,Kindle (2022) — najlżejszy i najbardziej kompa...,4.7,509.99,147
2,https://www.amazon.pl/kindle-paperwhite-signat...,Kindle Paperwhite Signature Edition (32 GB) – ...,4.8,869.99,753
3,https://www.amazon.pl/Kindle-2022/dp/B09SWTJZH...,Kindle (2022) — najlżejszy i najbardziej kompa...,4.7,509.99,147
4,https://www.amazon.pl/Kindle-regulacj%C4%85-po...,Kindle Oasis | z regulacją barwy podświetlenia...,4.5,999.99,58


In [86]:
df.dtypes

link          object
title         object
rate          object
price         object
nr_reviews    object
dtype: object

In [109]:
def fix_rate(x):
    if type(x) == str:
        if "& Darmowa dostawa" in x or "Suma:" in x:
            return np.NaN
        else:
            return x

In [111]:
df['rate'] = df['rate'].apply(fix_rate)

In [112]:
df

,link,title,rate,price,nr_reviews
0,https://www.amazon.pl/kindle-paperwhite-16-gb-...,Kindle Paperwhite (16 GB) – teraz z wyświetlac...,4.8,789.99,753
1,https://www.amazon.pl/Kindle-2022/dp/B09SWS16W...,Kindle (2022) — najlżejszy i najbardziej kompa...,4.7,509.99,147
2,https://www.amazon.pl/kindle-paperwhite-signat...,Kindle Paperwhite Signature Edition (32 GB) – ...,4.8,869.99,753
3,https://www.amazon.pl/Kindle-2022/dp/B09SWTJZH...,Kindle (2022) — najlżejszy i najbardziej kompa...,4.7,509.99,147
4,https://www.amazon.pl/Kindle-regulacj%C4%85-po...,Kindle Oasis | z regulacją barwy podświetlenia...,4.5,999.99,58
...,...,...,...,...,...
64,https://www.amazon.pl/przeno%C5%9Bny-wy%C5%9Bw...,"Czytnik e-booków, przenośny 15 cm wyświetlacz ...",1.9,472.92,2
65,https://www.amazon.pl/Tolino-4016621126872-Czy...,"Tolino Shine 3 4016621126872 Czytnik, Czarny, ...",4.2,NaN,848
66,https://www.amazon.pl/Czytnik-book%C3%B3w-Kr%C...,Czytnik e-booków Krüger & Matz Library 3S CART...,3.0,NaN,12
67,https://www.amazon.pl/PocketBook-Czytnik-book%...,PocketBook Czytnik e-booków InkPad 4 (wersja n...,,1 519.00,2


In [115]:
df.replace('', np.nan,inplace=True)

In [121]:
df.replace({None: np.nan},inplace=True)

In [151]:
df[['price', 'nr_reviews']] = df[['price', 'nr_reviews']].apply(lambda x: x.str.strip().str.replace(' ', '', 1) if x.dtype == 'O' else x)

## Final data frame

In [152]:
df

,link,title,rate,price,nr_reviews
0,https://www.amazon.pl/kindle-paperwhite-16-gb-...,Kindle Paperwhite (16 GB) – teraz z wyświetlac...,4.8,789.99,753
1,https://www.amazon.pl/Kindle-2022/dp/B09SWS16W...,Kindle (2022) — najlżejszy i najbardziej kompa...,4.7,509.99,147
2,https://www.amazon.pl/kindle-paperwhite-signat...,Kindle Paperwhite Signature Edition (32 GB) – ...,4.8,869.99,753
3,https://www.amazon.pl/Kindle-2022/dp/B09SWTJZH...,Kindle (2022) — najlżejszy i najbardziej kompa...,4.7,509.99,147
4,https://www.amazon.pl/Kindle-regulacj%C4%85-po...,Kindle Oasis | z regulacją barwy podświetlenia...,4.5,999.99,58
...,...,...,...,...,...
64,https://www.amazon.pl/przeno%C5%9Bny-wy%C5%9Bw...,"Czytnik e-booków, przenośny 15 cm wyświetlacz ...",1.9,472.92,2
65,https://www.amazon.pl/Tolino-4016621126872-Czy...,"Tolino Shine 3 4016621126872 Czytnik, Czarny, ...",4.2,NaN,848
66,https://www.amazon.pl/Czytnik-book%C3%B3w-Kr%C...,Czytnik e-booków Krüger & Matz Library 3S CART...,3.0,NaN,12
67,https://www.amazon.pl/PocketBook-Czytnik-book%...,PocketBook Czytnik e-booków InkPad 4 (wersja n...,NaN,1519.00,2
